In [3]:
import pysam
from tqdm import tqdm
import numpy as np
'''
dir="/Users/mhryansohn/Desktop/01.Workspace/01.Projects/03.AK1-PacBio/01.DNA/Public_Data/HG002"
hg002 = pysam.Ali
ak1 = pysam.AlignmentFile("chr20_500000-1000000_AK1.bam", 'rb')
a = next(ak1)
b = next(ak1)
a.tags # get all tags into list 
tag_name =  [tag[0] for tag in a.tags]

a.get_tag('MM')
a.get_tag('ML')
a.query_name
'''
'''
The continuous probability range of 0.0 to 1.0 is remapped to the discrete integers 0 to 255 
inclusively in the ML tag. The probability range corresponding to an integer N is N/256 to (N + 1)/256.
'''
'''
#Negative 일 경우
for pos, prob in list(a.modified_bases.values())[0]:
    if a.seq[pos-1: pos+1] != 'CG':
        print('fuck')
#Positive 일 경우
for pos, prob in list(b.modified_bases.values())[0]:
    if b.seq[pos: pos+2] != 'CG':
        print('fuck')
'''

for read in ak1:
    if read.has_tag('MM') and read.has_tag('ML'):
        print('fuck')
    
def get_read_level_cg(read_segment, pos):
    if read_segment.modified_bases != {}:
        # need to specify read.is_reverse == True or != True
        cg_ml = dict(list(read_segment.modified_bases.values())[0]) # read-level cytosine location and ML
        cg_met = list()
        if read.is_reverse:
            cg_ml = {key-1: value for key, value in cg_ml.items()}
        for k,v in read_segment.get_aligned_pairs(): # read-based coordinate and corresponding reference coordinate
            if v in pos:
                try:
                    cg_met.append(cg_ml[k]/256)
                except KeyError:
                    cg_met.append(np.nan)
        return np.array(cg_met)

def make_cg_matrix(alignmentfile, refcgtab, coordinate):
    '''alignmentfile = pysam.AlignmentFile("chr20_500000-1000000_AK1.bam", 'rb')'''
    cg_pos = list(map(lambda x: int(x), refcgtab.loc[coordinate]['CG_Pos'].rstrip(';').split(';')))
    chrom = coordinate.split(':')[0]
    start, end = int(coordinate.split(':')[1].split('-')[0]), int(coordinate.split(':')[1].split('-')[1])
    cg_matrix_list = list()
    for read in alignmentfile.fetch(chrom, start, end):
        cg_matrix_list.append(get_read_level_cg(read, cg_pos))
        print(read.query_name, cg_matrix_list)
    return np.array(cg_matrix_list)

dict(list(read3.modified_bases.values())[0]) # read-level cytosine location: ML
read3.get_aligned_pairs() # read position, reference coordinate

ref_10kb_cg = pd.read_table("10kb_CG_Information_hg38_primary_chrX.tab", index_col=0)

cg_pos = list(map(lambda x: int(x), ref_10kb_cg.loc['chr20:500000-510000']['CG_Pos'].rstrip(';').split(';')))

for i in cg_pos:
    


AttributeError: module 'pysam' has no attribute 'Ali'

modkit motif-bed /mnt/mone/Project/AK1_PacBio/Tools/Reference/GRCh38/DNA/hg38.analysisSet.fa CG 0 > hg38.analysisSet_CG.bed
modkit extract chr20_500000-1000000_AK1.bam chr20_500000-1000000_AK1_CpG.tsv --region chr20 --ref /mnt/mone/Project/AK1_PacBio/Tools/Reference/GRCh38/DNA/hg38.analysisSet.fa --include-bed hg38.analysisSet_CG.bed